### Twitter Scraper

In [ ]:
import json
import csv
import tweepy
import re
import pandas as pd

In [ ]:
## Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""

In [ ]:
## Authorize twitter, initialize tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)
api

In [ ]:
## Function to get tweets

def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)
    
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))
    
    #transform the tweepy tweets into a 2D array that will populate the csv	
    outtweets = [[screen_name, tweet.id_str, tweet.created_at, tweet.text, tweet.retweet_count, tweet.favorite_count] for tweet in alltweets]
    tweets_df=pd.DataFrame(outtweets)
    return tweets_df

    pass

#### Read in the list of CEOs for analysis

In [ ]:
ceo_list = pd.read_csv("ceo.csv")
ceo_list.head()

In [ ]:
ceo_tweets = pd.DataFrame(columns=['username', 'id', 'date', 'tweet', 'retweets','likes'])
                                   
for handle in ceo_list["Twitter Handle of CEO"]:
    ceo_tweets_temp = get_all_tweets("@"+handle)
    ceo_tweets_temp.columns = ['username', 'id', 'date', 'tweet','retweets','likes']
    
    ceo_tweets = ceo_tweets.append(ceo_tweets_temp, ignore_index=True)

In [ ]:
ceo_tweets.to_csv('ceo_tweets.csv')